# 송파구 안전센터 관할동 시각화 코드(folium)

In [1]:
!pip freeze > requeriments.txt

In [2]:
%%capture
if 'google.colab' in str(get_ipython()):
  !apt install libspatialindex-dev
  !pip install fiona shapely pyproj rtree mapclassify
  !pip install geopandas

In [3]:
import folium
import geopandas as gpd
import pandas as pd
from folium.plugins import MarkerCluster
from math import radians, sin, cos, sqrt, asin
import numpy as np


In [4]:
# 2023년 7월 행정동 기준.
df_bdong = gpd.read_file('/content/drive/MyDrive/세미프로젝트/data/HangJeongDong_ver20230701.json')
df_bdong.head()

,adm_nm,adm_cd,adm_cd2,sgg,sido,sidonm,temp,sggnm,adm_cd8,geometry
0,서울특별시 종로구 사직동,1101053,1111053000,11110,11,서울특별시,종로구 사직동,종로구,11010530,"POLYGON ((126.97689 37.57565, 126.97580 37.575..."
1,서울특별시 종로구 삼청동,1101054,1111054000,11110,11,서울특별시,종로구 삼청동,종로구,11010540,"POLYGON ((126.98269 37.59507, 126.98106 37.595..."
2,서울특별시 종로구 부암동,1101055,1111055000,11110,11,서울특별시,종로구 부암동,종로구,11010550,"POLYGON ((126.97585 37.59656, 126.97401 37.596..."
3,서울특별시 종로구 평창동,1101056,1111056000,11110,11,서울특별시,종로구 평창동,종로구,11010560,"POLYGON ((126.97507 37.63139, 126.97179 37.632..."
4,서울특별시 종로구 무악동,1101057,1111057000,11110,11,서울특별시,종로구 무악동,종로구,11010570,"POLYGON ((126.96067 37.58080, 126.95878 37.581..."


In [5]:
cond = df_bdong['sggnm'] == "송파구"
gdf_test = df_bdong[cond]
gdf_test = gdf_test[['temp', 'geometry']]


gdf_test['temp'] = gdf_test['temp'].str[4:]
gdf_test.rename(columns={'temp' : 'EMD_KOR_NM'}, inplace=True)
gdf_test['SIG_KOR_NM'] = '송파구'

gdf_test.head(2)

,EMD_KOR_NM,geometry,SIG_KOR_NM
379,풍납1동,"POLYGON ((127.12321 37.53853, 127.12161 37.539...",송파구
380,풍납2동,"POLYGON ((127.12126 37.53311, 127.11982 37.533...",송파구


In [6]:
# 각 안전센터 관할 구역 폴리곤 시각화
df_Songpa = gdf_test[gdf_test['EMD_KOR_NM'].isin(['오금동', '가락2동', '문정1동', '장지동'])] # 서울 송파 소방서
df_Geoyeo = gdf_test[gdf_test['EMD_KOR_NM'].isin(['거여1동', '거여2동', '마천1동', '마천2동'])] # 거여 안전센터
df_Bangi = gdf_test[gdf_test['EMD_KOR_NM'].isin(['방이1동', '방이2동', '풍납1동','풍납2동', '오륜동'])] # 방이 안전센터
df_Jamsil = gdf_test[gdf_test['EMD_KOR_NM'].isin(['잠실3동', '잠실4동', '잠실6동', '삼전동','석촌동', '송파1동'])] # 잠실 안전센터
df_Jonghap = gdf_test[gdf_test['EMD_KOR_NM'].isin(['잠실본동', '잠실1동', '잠실2동','잠실5동', '잠실7동'])] # 종합운동장 안전센터
df_Garak = gdf_test[gdf_test['EMD_KOR_NM'].isin(['가락본동', '가락1동', '문정2동', '송파2동'])] # 가락 안전센터

m = folium.Map(location=(37.505537, 127.115318), tiles='OpenStreetMap', zoom_start=11)

list = ['Songpa', 'Geoyeo', 'Bangi', 'Jamsil', 'Jonghap', 'Garak']
layer_list = ['layer_Songpa', 'layer_Geoyeo', 'layer_Bangi', 'layer_Jamsil', 'layer_Jonghap', 'layer_Garak']
# 시각화 할 경계선 색상 지정
colors = ['red', 'blue', 'black', 'beige', 'darkblue', 'yellow']
# 데이터프레임들을 리스트에 담기
dataframes = [df_Songpa, df_Geoyeo, df_Bangi, df_Jamsil, df_Jonghap, df_Garak]


for i in range(0,6):
  layer_list[i] = folium.FeatureGroup(name='송파 소방서 관할 권역', control=False, show=True)
  for idx, row in dataframes[i].iterrows():
      polygon_wkt = row['geometry']
      t1 = folium.GeoJson(polygon_wkt, style_function=lambda feature, color=colors[i]: {
                            'fillColor': color,
                            'fillOpacity': 0.5,
                            'color': color,
                            'weight': 1,
                            'opacity': 1
                        })
      t1.add_to(layer_list[i])
  layer_list[i].add_to(m)


# 비상 소화장치 점 / 안전센터 위치 찍기

In [7]:

# installed_equip = pd.read_csv('/content/drive/MyDrive/세미프로젝트/data/비상소화장치.csv', encoding='euc-kr')
# installed_equip.set_index('순번', inplace=True)  # set_index 메서드는 inplace=True로 설정하여 데이터프레임을 변경
# installed_equip = installed_equip[['설치지역', '안전센터', '사용구분', '경위도좌표X', '경위도좌표Y']]  # 필요한 열만 선택


# equip_group = folium.FeatureGroup(name="비상소화장치 위치", show=False)

# # 비상 소화장치 위치에 대한 점 추가
# for idx, row in installed_equip.iterrows():
#     folium.CircleMarker(
#         location=[row['경위도좌표Y'], row['경위도좌표X']],
#         radius=9,  # 점의 크기
#         color='red' if row['사용구분'] == "고장(사용불)" else 'green',  # 색상
#         fill=True,
#         fill_color='red' if row['사용구분'] == "고장(사용불)" else 'green',  # 채우기 색상
#         fill_opacity=0.7,
#         popup=folium.Popup(f"<b>설치지역:</b> {row['설치지역']}<br><b>안전센터:</b> {row['안전센터']}<br><b>사용구분:</b> {row['사용구분']}", max_width=300)
#     ).add_to(equip_group)

#     folium.Circle(
#         location=[row['경위도좌표Y'], row['경위도좌표X']],
#         radius=50,  # 반경
#         color='yellow',  # 원의 테두리 색상
#         fill=False,  # 원 내부를 채우지 않음
#         opacity=0.5,  # 투명도
#     ).add_to(equip_group)

#     equip_group.add_to(m)


center_group = folium.FeatureGroup(name="안전센터 위치", show=False)


# 안전센터 마커 위치
latitude = [37.5102514,
37.5027107,
37.5120841,
37.4813193,
37.4979534,
37.5068859,
37.4969882,
37.5201652,
37.5341551,
37.5148527,
37.4778753,
37.489969,
37.503659,
37.5062783,
37.5134856,
37.5132807,
37.4932304,
37.5350689]

longitude = [127.1257318,
127.0927721,
127.0821495,
127.1267464,
127.124438,
127.1280993,
127.1482966,
127.1123922,
127.1170847,
127.0802537,
127.1458547,
127.1111015,
127.1038721,
127.1093064,
127.1048655,
127.0942379,
127.1323757,
127.1204641]

name = ['방이지구대',
'삼전지구대',
'잠실지구대',
'문정지구대',
'가락지구대',
'오금파출소',
'마천파출소',
'신천파출소',
'풍납파출소',
'잠실여름파출소',
'위례파출소',
'문정2치안센터',
'석촌치안센터',
'송파1치안센터',
'월드치안센터',
'잠실5치안센터',
'가락2치안센터',
'풍납1치안센터']


for i in range(0, 18):
  folium.Marker(
      location=[latitude[i], longitude[i]],
      popup=folium.Popup("<b>"+name[i]+"</b>", max_width=300),
      icon=folium.Icon( icon_size=(28,28)),
  ).add_to(center_group)

  center_group.add_to(m)




# 소화전 점찍기

In [8]:
# 화재구역 점 찍기
fired = pd.read_csv('/content/drive/MyDrive/세미프로젝트/data/화재출동 현황2.csv', encoding='euc-kr')
fired = fired[['la', 'longitude']]  # 필요한 열만 선택
fired.dropna(inplace=True)
fired_group = folium.FeatureGroup(name="화재발생한 주택 위치", show=False)

# 화재구역 점 찍기 및 반경 50m 빨간색 원 그리기
for idx, row in fired.iterrows():
    # 화재난 위치에 대한 점 추가
    folium.CircleMarker(
        location=[row['la'], row['longitude']],
        radius=5,  # 점의 크기
        color='black',  # 점의 색상
        fill=True,
        fill_color='black',  # 점 내부의 색상
        fill_opacity=0.7,
    ).add_to(fired_group)

    # # 반경 50m의 빨간색 원 그리기
    # folium.Circle(
    #     location=[row['la'], row['longitude']],
    #     radius=50,  # 반경
    #     color='red',  # 원의 테두리 색상
    #     fill=False,  # 원 내부를 채우지 않음
    #     opacity=0.5,  # 투명도
    # ).add_to(fired_group)

    fired_group.add_to(m)


In [ ]:
# Haversine 공식을 이용하여 두 지점 간의 거리 계산
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # Decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r * 1000  # in meters

# 소화전 점찍기
fire_hydrant = pd.read_csv('/content/drive/MyDrive/세미프로젝트/data/소방용수2.csv', encoding='euc-kr')
fire_hydrant.set_index('순번', inplace=True)  # set_index 메서드는 inplace=True로 설정하여 데이터프레임을 변경
fire_hydrant['<=50m'] = np.nan
fire_hydrant = fire_hydrant[['사용구분', '안전센터', '경위도X', '경위도Y', '<=50m']]  # 필요한 열만 선택

# emergency_device = installed_equip[['경위도좌표Y', '경위도좌표X']]  # 필요한 열 선택


# 화재 장소 50m 반경 내의 소화전 구분
for _, fired_row in fired.iterrows():
    for idx, fire_hydrant_row in fire_hydrant.iterrows():
        distance = haversine(fired_row['longitude'], fired_row['la'], fire_hydrant_row['경위도X'], fire_hydrant_row['경위도Y'])
        if distance <= 50:  # 50m 이내인 경우
            fire_hydrant.loc[idx, '<=50m'] = 1


# print(fire_hydrant['<=50m'].value_counts())
fire_hydrant = fire_hydrant[fire_hydrant['<=50m'] == 1]

# MarkerCluster 생성
marker_cluster = MarkerCluster(name="소화전 위치", show=False).add_to(m)

# 소화전 위치에 마커 추가
for idx, row in fire_hydrant.iterrows():
    if row['사용구분'] == '고장(사용불)':
      color = 'red'
    else :
      color = 'green'
    folium.Marker(
        [row['경위도Y'], row['경위도X']],
        icon=folium.Icon(color=color),
        popup=f"<b>사용구분:</b> {row['사용구분']}<br><b>안전센터:</b> {row['안전센터']}<br><b>경위도좌표Y:</b> {row['경위도Y']}<br><b>경위도좌표X:</b> {row['경위도X']}",
        max_width=500
    ).add_to(marker_cluster)



In [ ]:


m.save('map.html')

m